In [1]:
PORT = 9000
#store url of peer nodes
peer_nodes = [8070,8080,8090,9000]
peer_nodes.remove(PORT)

In [2]:
from flask import Flask #flask is similar to django
from flask import request
import json
import requests
import hashlib
import datetime as date

In [3]:
node = Flask(__name__)

In [4]:
class Block:
    def __init__(self,index,timestamp,data,nonce,previous_hash,cur_hash=""):
        
        self.index = index
        self.timestamp = str(timestamp)
        self.data = data
        self.nonce = nonce
        self.previous_hash = previous_hash
        self.hash = cur_hash 
        
    def hash_block(self):
        #all the data to hash upon, note: it needs to be encoded to utf-8
        prehash = (str(self.index)+str(self.timestamp)+str(self.data)+str(self.previous_hash)+str(self.nonce)).encode('utf-8') 
        result = hashlib.sha256(prehash)
        return str(result.hexdigest())
    
    def proof_of_work(self):
        global blockchain
        global pending_peer_requests
        print("finding PoW")
        self.nonce = 0
        x = self.hash_block()
        while(x[:2]!='00'):
            self.nonce = self.nonce+1
            x = self.hash_block()
        self.hash = x
        print(self.index,self.timestamp,self.data,self.previous_hash,self.nonce,self.hash)

In [5]:
address = "miner_1"


In [6]:
blockchain = []
pending_peer_requests = []

In [7]:
@node.route('/getchain', methods=['GET'])
def get_chain_json():
    global blockchain
    global pending_peer_requests
    #convert blocks into dictionaries so that they can be sent as json objects
    chain_to_send = []
    for block in blockchain:
        chain_to_send.append({
            "index": block.index,
            "timestamp": block.timestamp,
            "data" : block.data,
            "nonce" : block.nonce,
            "previous_hash" : block.previous_hash,
            "hash" : block.hash
        })
    
    #convert the list of dicts into json object
    chain_to_send = json.dumps(chain_to_send)
    return chain_to_send


def send_chain():
    
    
    global blockchain
    global pending_peer_requests
    print("chain sent successfully")
    return  get_chain_json()

#check if computed hash on each block is equal to hash stored in it
def verify_chain(chain):
    global blockchain
    global pending_peer_requests
    for block in chain[1:]:
        if block.hash_block() != block.hash:
            return False
    return True

#obtain new longest chain from peers using their localhost urls
#obtain new longest chain from peers using their localhost urls
@node.route('/updateChain', methods=['GET'])
def find_new_longest_chain():
    
    global blockchain
    global pending_peer_requests
    for peer_port in peer_nodes:
        node_url = "http://localhost:" + str(peer_port)
        peer_chain_json = requests.get(node_url + "/getchain").content
        if(peer_chain_json is None):
            print("pee_chain_json is None")
        peer_chain_dict = json.loads(peer_chain_json)
        
        peer_chain = []
        for b in peer_chain_dict:
            block = Block(b["index"],b["timestamp"],b["data"],b["nonce"],b["previous_hash"],b["hash"])
            peer_chain.append(block)
        
        if verify_chain(peer_chain) and len(peer_chain) > len(blockchain):
            print("Blockchain updated")
            blockchain = peer_chain
        
            
    print(blockchain)
    return get_chain_json()

In [8]:
def create_genesis_block():
    global blockchain
    global pending_peer_requests
    block = Block(0,str(date.datetime.now()), "NoTransactionsYet",0 , "NoPreviousHashYet")
    block.hash = block.hash_block()
    blockchain.append(block)

In [9]:
#miner creates genesis block:

create_genesis_block()
print("created genesis block","hash: ",blockchain[0].hash)

created genesis block hash:  c64b958390bebecf84e44327b226b6fd36f4c01b4e240dc5af377d315ad299eb


In [10]:
def getPendingTransactions():
    global blockchain
    global pending_peer_requests
    for peer_port in peer_nodes:
        node_url = "http://localhost:" + str(peer_port)
        peer_requests_json = requests.get(node_url + "/getPendingTransactions").content
        peer_requests = json.loads(peer_requests_json)
        
        pending_peer_requests.append(peer_requests)

In [11]:
@node.route('/createBlock', methods=['GET'])
def create_block():
    global blockchain
    global pending_peer_requests
    find_new_longest_chain()
    index = blockchain[-1].index +1
    timestamp = str(date.datetime.now())
    getPendingTransactions()
    data = pending_peer_requests
    pending_peer_requests = []
    previous_hash = blockchain[-1].hash
    
    new_block = Block(index,timestamp,data,0,previous_hash)
    new_block.proof_of_work()
    
    print(new_block.index,new_block.timestamp,new_block.data,new_block.previous_hash,new_block.nonce,new_block.hash)
    blockchain.append(new_block)
    
    print(blockchain)
    return "block created successfully"

In [12]:
node.run(host='0.0.0.0', port=PORT, threaded=True, debug=False)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://0.0.0.0:9000/ (Press CTRL+C to quit)
127.0.0.1 - - [03/Nov/2020 09:23:34] "GET /getchain HTTP/1.1" 200 -
127.0.0.1 - - [03/Nov/2020 09:23:52] "GET /createBlock HTTP/1.1" 200 -


finding PoW
1 2020-11-03 09:23:52.639956 [[{'to': 'e', 'amount': 10, 'from': 'user1_addr'}, {'to': 'f', 'amount': 10, 'from': 'user1_addr'}, {'to': 'g', 'amount': 10, 'from': 'user1_addr'}], [], []] c64b958390bebecf84e44327b226b6fd36f4c01b4e240dc5af377d315ad299eb 135 0008968996be1d1750e3e2a486ee9e3fc0145a4669e600a65d67c201f03603de
1 2020-11-03 09:23:52.639956 [[{'to': 'e', 'amount': 10, 'from': 'user1_addr'}, {'to': 'f', 'amount': 10, 'from': 'user1_addr'}, {'to': 'g', 'amount': 10, 'from': 'user1_addr'}], [], []] c64b958390bebecf84e44327b226b6fd36f4c01b4e240dc5af377d315ad299eb 135 0008968996be1d1750e3e2a486ee9e3fc0145a4669e600a65d67c201f03603de
[<__main__.Block object at 0x7fd55d168320>, <__main__.Block object at 0x7fd55d1326d8>]


127.0.0.1 - - [03/Nov/2020 09:23:57] "GET /getchain HTTP/1.1" 200 -
127.0.0.1 - - [03/Nov/2020 09:24:02] "GET /getchain HTTP/1.1" 200 -
127.0.0.1 - - [03/Nov/2020 09:24:08] "GET /getchain HTTP/1.1" 200 -
